# OpenCAPI course lab1, Introduction to oc-accel

In this lab, we will briefly introduce the workflow of the oc-accel acceleration environment.
The goal is to help you understand the process of starting a so-called "Hardware/software co-simulation" using oc-accel's helloworld tutorial.

In this notebook we assume you are running from the virtual machine that is provided with the course, and have already run the installation script.
If you wish to run the notebook from a different environment, you need to install the dependencies yourself, modify the path where oc-accel is installed, and perform the necessary configuration in oc-accel (e.g. configure the location of ocse).


# Overview of oc-accel
Before we begin, it is important to realize that OpenCAPI is based on the IBM POWER platform. 
However, developing and testing is done on x86 machines, because that is the platform supported by the software of FPGA vendors.
So the OpenCAPI software libraries support IBM POWER for normal operation (using a real FPGA), _and_ have a simulation mode to simulate an FPGA design on x86.

## Components
In the OpenCAPI acceleration environment, several components work together;
- __oc-accel__ Framework containing scripts, tools, base designs and examples to create OpenCAPI accelerators.
- __ocse__ Simulation environment that allows `oc-accel` to execute software as if it was running on a POWER system with the FPGA design under test.
  - __libocxl__ Is the library that allows programs to interface with OpenCAPI designs (either in simulation or actual hardware). It is included in the Linux kernel, but the simulation version of the library is located in `ocse`.
- __Vivado__ Is the FPGA vendor toolchain from Xilinx. It can perform circuit simulation (used by the simulation environment) and synthesize a design to an actual FPGA.
  Vivado can also be used to inspect the behavior of all the internals of your design after running a simulation (waveform inspection).

## Documentation
This notebook is only a small tutorial showing how to run the helloworld example. The full documentation can be found [here](https://opencapi.github.io/oc-accel-doc/).

## Installation
As stated, we assume you are running the VM provided by the OpenCAPI course and have run the installation procedure from [this repository](https://github.com/joosthooz/fletcher-oc-accel-installscripts), which is included in [the course toplevel repository](https://github.com/abs-tudelft/opencapicourse_toplevel).

# Description of the workflow
oc-accel provides Makefiles with various targets for the steps needed to build an accelerator design.
In addition, there is a python script, `ocaccel_workflow.py` that will guide the user through these steps using a menu-based configuration.

## Steps
- __Configure__ During configuration, the user chooses which FPGA board to use, general OpenCAPI parameters, and which particular "action" the accelerator should perform. An action consists of a hardware design and usually also includes some example code to demonstrate how to call this accelerator from software. Users can add actions of their own, or modify an action to their needs.
- __Build model__ This step builds the simulation model based on the action's hardware design. It includes compiling all of the hardware sources using the circuit simulator's toolchain. 
- __Software compilation__ The action's software is compiled and linked against the needed libraries such as libocxl.
- __Run simulation__ The circuit simulation is started, and connected to the simulation environment. This will forward the library calls to the simulation. This way, the executed happens in almost the exact same way as if the program would be running on a POWER machine with an FPGA attached. The timing behavior will be quite different, as the simulation is orders of magnitude slower than the real FPGA.
- __waveform inspection__ All the design's internal behavior is stored during the simulation. This step opens a waveform viewer to look at these signals. The user can choose which signals to look at.
- __Synthesis__ If the accelerator has been verified to be working properly in simulation, a "bitstream" can be generated for the FPGA. This bitstream can then be loaded onto the FPGA by the POWER machine in which it resides.

In the following cell, we will use the scripting facilities of oc-accel to perform all the needed steps for the `HLS_helloworld` example. After this, we encourage you to perform the steps as explained in [this page in the manual](https://opencapi.github.io/oc-accel-doc/user-guide/2-run-helloworld/), and verify that the results are the same.

In [ ]:
import os
WORKSPACE="/home/opencapi/workspace"
ocaccel_path=os.path.join(WORKSPACE, "OpenCAPI/oc-accel")
ocaccel_defconfig_path=os.path.join(ocaccel_path, "defconfig/")

# Add the configuration file we have prepared for this lab to oc-accel
!cp hls_helloworld.defconfig {ocaccel_defconfig_path}

In [ ]:
# In this example, we run HLS_helloworld using oc-accel
# Go into the oc-accel working directory
%cd {ocaccel_path}

!make -s hls_helloworld.defconfig && ./ocaccel_workflow.py --no_configure -t {ocaccel_path}/actions/hls_helloworld_1024/tests/hw_test.sh



In [ ]:
!cat actions/hls_helloworld_1024/tests/hw_test.sh.log